<a href="https://colab.research.google.com/github/MateoProjects/Schneider-Electric-European-Hackathon/blob/main/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Schneider Electric European Hackathon

**Authors** : Anton ... , Ramon Mateo

**Description**: To do

## Load Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install keras-cv
!cp "poner path del csv" .
!cp "path del zip con imagenes" .
!unzip -qq MAMe_data_256.zip
!unzip -qq MAMe_metadata.zip
!rm MAMe_data_256.zip
!rm MAMe_metadata.zip
!rm MAMe_dataset.csv
!cp /content/drive/MyDrive/DL/L2/MAMe_dataset.csv .


## Imports

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import random
import cv2
import keras_cv
from PIL import Image
from skimage import io
from tensorflow.keras import utils
from tensorflow.keras.utils import Sequence
from tensorflow import keras
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras import applications
from skimage.transform import rotate, rescale
from keras.models import Sequential, Model 

## Constants

In [ ]:
NUM_LABELS = 20

## Prepare data

In [ ]:
input_dir_train = ""
target_dir_train = ""
input_dir_test = ""
target_dir_test = ""


## Data Generator

In [ ]:
class DataGenerator(Sequence):

  def __init__(self, x_names, y_names, batch_size):
    
    self.x_values = x_names.copy()
    self.y_values =   y_names.copy()
    self.num_imgs = len(x_names)
    self.batch_size = batch_size

  def __getitem__(self, index):

    # position of the batch in the sequence

    aux_index = index * self.batch_size

    x = [] # numpy array with shape (batch_size, input_height, input_width, input_channel)
    y = [] # numpy array with shape (batch_size, num_classes)
    

    for _ in range(self.batch_size):

      img_name = self.x_values[aux_index]
      img = cv2.imread('/content/data_256/' + img_name)
      x.append(img)
      y.append(self.y_values[aux_index])
      aux_index += 1

    x = np.array(x)
    y = np.array(y)
    return x, y

  def on_epoch_end(self):
    num = random.randint(0,1000)
    random.Random(num).shuffle(self.x_values)
    random.Random(num).shuffle(self.y_values)

  def __len__ (self):
    # return the number of batches the generator can produce
    return (self.num_imgs) // self.batch_size

## Models

In [ ]:
"""
VGG16
"""

def get_vgg16(img_width=256, img_height=256):
  """ 
  VGG16 for image classification 
  @return Return vgg16 with more dense layers
  """
  model = applications.vgg16.VGG16(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
  #model.trainable = True
  for layer in model.layers[:10]:
    layer.trainable = False

  #Adding custom Layers 
  x = model.output
  x = keras.layers.Flatten()(x)
  x = keras.layers.Dense(4096, activation="relu")(x)
  x = keras.layers.Dropout(0.2)(x)
  x = keras.layers.Dense(1024, activation="relu")(x)
  predictions = keras.layers.Dense(NUM_LABELS, activation="softmax")(x)
  return predictions


def get_resnet50(img_width=256, img_height=256):
  model = applications.resnet50.Resnet50(weights="imagenet", include_top=False, input_shape = (img_width, img_height, 3))
  return model


models = {"resnet50": get_resnet50, "vgg16": get_vgg16}

In [ ]:
#@title Select Model and Hyper parameters { run: "auto", display-mode: "form" }
model_name = "resnet50" #@param ["resnet50", "vgg6"]
EPOCHS = 50 #@param {type: "integer"}
BATCHSIZE = 16 #@param {type: "integer"}
IMG_SIZE = 256 #@param [128,256,512, 1024]
DATA_AUGMENTATION = True #@param {type: "boolean"}

model = models[model_name](IMG_SIZE, IMG_SIZE)

if DATA_AUGMENTATION:
  # concatenamos paths de nuestras imagenes con DA con las de la base
  pass


## Training

In [ ]:
#@title Push play for train { run: "auto", display-mode: "form" }

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5,verbose=0,mode="auto",restore_best_weights=True)
metric_f1 =  tfa.metrics.F1Score(num_classes, average=None, , name='f1_score', dtype: tfa.types.AcceptableDTypes = None)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy', metric_f1])
data = DataGenerator()
mdl_fit = model.fit(data, epochs = EPOCHS, validation_data = data_val, batch_size=BATCH_SIZE,callbacks = [early_stopping])


## Plots


In [ ]:
# plot the loss
plt.plot(mdl_fit.history['loss'], label='train loss')
plt.plot(mdl_fit.history['val_loss'], label='val loss')
plt.legend()
plt.savefig("model_"+ model_name + "data_epochs_" +str(EPOCHS)+"_batchSize_"+ str(BATCHSIZE)+'_LossVal_loss')
plt.show()
# plot the accuracy
plt.plot(mdl_fit.history['accuracy'], label='train acc')
plt.plot(mdl_fit.history['val_accuracy'], label='val acc')
plt.legend()
plt.savefig("model_" + model_name +"data_epochs_" +str(EPOCHS)+"_batchSize_"+ str(BATCHSIZE)+'_AccVal_acc')
#save model to disk
plt.show()

## F1 Score

In [ ]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

f1 = (2*tp)/(2*tp + fp + fn)

print("True Positive:" , tp)
print("True Negative:", tn)
print("False positve:", fp)
print("False negative:", fn)
print("F1 Score: ", f1)

## Data Augmentation

Note: This code was executed in other script. Code below is only for show how Data augmentation was do it. You can find the main script on the same repository

### Simple Data Augmentation

Generating new images rotating the original image.

In [ ]:
import os
import numpy as np
from skimage import io, transform

input_path = "Training_Data_p1\\ISBI2016_ISIC_Part1_Training_Data\\"
target_path = "Training_GroundTruth_p1\\ISBI2016_ISIC_Part1_Training_GroundTruth\\" # esto debe ser modificado para que lea el csv o generar uno nuevo
output_path_img = "Training_Data_p1_DA\\"
output_path_ground = "Training_GroundTruth_p1_DA\\" # lo mismo que lo de antes. Mi idea es generar un csv nuevo con las imagenes aplicando el DA y luego se concatenan al original


def get_paths(input_dir, target_dir):
  input_img_paths = sorted(
      [
          os.path.join(input_dir, fname)
          for fname in os.listdir(input_dir)
          if fname.endswith(".jpg")
      ]
  )
  target_img_paths = sorted(
      [
          os.path.join(target_dir, fname)
          for fname in os.listdir(target_dir)
          if fname.endswith(".png") and not fname.startswith(".")
      ]
  )
  return input_img_paths, target_img_paths

def data_augmentation(input_path, target_path, output_path, output_target_path):
    for path_in, path_targ in zip(input_path, target_path):
        name = path_in.split("\\")[-1].split(".")[0]
        img = io.imread(path_in)
        target = io.imread(path_targ)
        img_90 = transform.rotate(img,90)
        target_90 = transform.rotate(target,90)
        img_180 = transform.rotate(img, 180)
        target_180 = transform.rotate(target, 180)
        io.imsave(output_path + name + "_90.jpg", img_90)
        io.imsave(output_path + name + "_180.jpg", img_180)
        io.imsave(output_target_path + name + "_90.png", target_90) # esta linea debe ser sustituida para añadir al csv el label
        io.imsave(output_target_path + name + "_180.png", target_180) #lo mismo que la anterior 

        

path_inp, path_targ = get_paths(input_path, target_path)
data_augmentation(path_inp, path_targ, output_path_img, output_path_ground)

### Data Augmentation using CutMix, MixUp, and RandAugment image augmentation with KerasCV

Link: https://keras.io/guides/keras_cv/cut_mix_mix_up_and_rand_augment/


In [ ]:
def visualize_dataset(dataset, title):
    plt.figure(figsize=(6, 6)).suptitle(title, fontsize=18)
    for i, samples in enumerate(iter(dataset.take(9))):
        images = samples["images"]
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[0].numpy().astype("uint8"))
        plt.axis("off")
    plt.show()

rand_augment = keras_cv.layers.RandAugment(
    #value_range describes the range of values covered in your images
    value_range=(0, 255),
    #augmentations_per_image is an integer telling the layer how many augmentations to apply to each individual image
    augmentations_per_image=3,
    # magnitude is a value between 0 and 1, describing the strength of the perturbations applied
    magnitude=0.3,
    # (Optional) magnitude_stddev allows magnitude to be randomly sampled from a distribution with a standard deviation of magnitude_stddev
    magnitude_stddev=0.2,
    # (Optional) rate indicates the probability to apply the augmentation applied at each layer.
    rate=0.5,
)
data_ra = rand_augment(data)

visualize_dataset(data, "After rand_augment")



In [ ]:
cut_mix = keras_cv.layers.CutMix()
mix_up = keras_cv.layers.MixUp()


def cut_mix_and_mix_up(samples):
    samples = cut_mix(samples, training=True)
    samples = mix_up(samples, training=True)
    return samples

data_cm = cut_mix_and_mix_up(samples)
visualize_dataset(data_cm, "After CutMix and MixUp")
